In [1]:
import os
import sys
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
import matplotlib.pyplot as plt
from Naive_DAE import Naive_DAE, Conv_DAE
from AE_Stats import generate_stats
from load_data_fn import load_data

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
dt,dt_loc = load_data(1)

In [3]:
def AE_MSE(data,pred):
    return torch.mean(torch.sum((data-pred)**2,dim=1))

def weighted_AE_MSE(data, pred):
    return torch.mean(torch.sum((data[:,0:48])**2,dim=1)*torch.sum((data-pred)**2,dim=1))

def dif_weighted_AE_MSE(data, pred,alpha = 1):
    return torch.mean(((torch.sum(data[:,0:48]**2,dim=1)**0.5)*alpha+1)*(torch.sum((data-pred)**2,dim=1)))

def log_dif_weighted_AE_MSE(data, pred,alpha = 1):
    return torch.mean(torch.log((torch.sum(data[:,0:48]**2,dim=1)**0.5)*alpha+1)*(torch.sum((data-pred)**2,dim=1)))

In [25]:
num_epochs = 4
lr = 0.00025
loss = dif_weighted_AE_MSE

In [5]:
size_train =200000
size_test = 40000
train_loc = torch.unsqueeze(dt_loc[0:size_train],dim=1)
test_loc = torch.unsqueeze(dt_loc[-size_test:],dim=1)
# train = torch.vstack([train,largest[0:1000]] )
train_loc=train_loc[torch.randperm(train_loc.size()[0])]
test_loc=test_loc[torch.randperm(test_loc.size()[0])]
train_loc_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(train_loc)),
    batch_size=200,
    shuffle=False
)

test_loc_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(test_loc)),
    batch_size=200,
    shuffle=False
)

train = torch.unsqueeze(dt[0:size_train],dim=1)
test = torch.unsqueeze(dt[-size_test:],dim=1)
# train = torch.vstack([train,largest[0:1000]] )
train=train[torch.randperm(train.size()[0])]
test=test[torch.randperm(test.size()[0])]
train_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(train)),
    batch_size=200,
    shuffle=False
)

test_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(test)),
    batch_size=200,
    shuffle=False
)

In [26]:
###### fine-tune autoencoder
#batch 500


# model_5 = Conv_DAE([48,250,100,16])

optimizer = optim.Adam(model_5.parameters(), lr,weight_decay=5e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.5)

all_test_losses = []
all_train_losses = []
# train
running_loss = float("inf")
for epoch in range(num_epochs):
    
    losses = []
    
    for i, data_list in enumerate(train_dl_flat):
        data = data_list[0]
        v_pred = torch.squeeze(model_5(data))
        
        batch_loss = loss(data, v_pred,alpha = 0.05) # difference between actual and reconstructed   
        
        
        all_train_losses.append(batch_loss.item())
        losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    scheduler.step(batch_loss)
    data_test = test
    test_pred = torch.squeeze(model_5(data_test))
    batch_test = loss(data_test, test_pred)
    running_loss = np.mean(losses)
    running_test_loss = batch_test.item()
    print('Epoch {}, lr {}'.format(
        epoch, optimizer.param_groups[0]['lr']))
    
    print(f"Epoch {epoch}: Train {running_loss}, Test {running_test_loss}")
generate_stats(model_5(test),test,test_loc)


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 307200000000 bytes. Error code 12 (Cannot allocate memory)

In [12]:
next(iter(train_dl_flat))

[tensor([[ 7.7088e-01, -1.5858e-01, -1.6369e-01,  ..., -1.5346e-01,
          -1.5054e-01, -1.5277e-01],
         [ 5.3968e-01,  2.0050e+00, -5.1730e-02,  ..., -6.5281e-02,
           2.4527e-01,  5.7730e-01],
         [-1.5390e-01, -1.5858e-01, -1.6369e-01,  ..., -1.5346e-01,
           1.9093e-02, -1.5277e-01],
         ...,
         [-1.5390e-01, -1.5858e-01, -1.6369e-01,  ..., -1.5346e-01,
          -1.5054e-01, -1.5277e-01],
         [ 2.2935e-04,  7.8478e-03,  4.2519e-03,  ...,  2.5805e-01,
          -6.5725e-02, -6.5163e-02],
         [-1.5390e-01, -1.5858e-01, -1.6369e-01,  ..., -1.5346e-01,
          -1.5054e-01, -1.5277e-01]])]

In [23]:
torch.squeeze(model_5(test[0]))

tensor([0.0151, 0.0223, 0.0231, 0.0222, 0.0217, 0.0187, 0.0321, 0.0258, 0.0185,
        0.0195, 0.0303, 0.0226, 0.0154, 0.0217, 0.0217, 0.0157, 0.0189, 0.0117,
        0.0219, 0.0211, 0.0213, 0.0181, 0.0141, 0.0168, 0.0205, 0.0198, 0.0367,
        0.0188, 0.0233, 0.0146, 0.0169, 0.0259, 0.0187, 0.0234, 0.0224, 0.0181,
        0.0209, 0.0164, 0.0244, 0.0166, 0.0176, 0.0215, 0.0199, 0.0187, 0.0245,
        0.0285, 0.0191, 0.0198], grad_fn=<SqueezeBackward0>)

In [21]:
test[20]

tensor([[-0.1539, -0.1586, -0.1637, -0.1626, -0.1643, -0.1584, -0.1642, -0.1666,
         -0.1621, -0.1621, -0.1590, -0.1614, -0.1505, -0.1545, -0.1546, -0.1472,
         -0.1641, -0.1621, -0.1568, -0.1644, -0.1612, -0.1625, -0.1638, -0.1639,
         -0.1588, -0.1612, -0.1665, -0.1573, -0.1583, -0.1476, -0.1555, -0.1461,
         -0.1590, -0.1623, -0.0508, -0.1612, -0.1641, -0.1585, -0.1601, -0.1620,
         -0.1597, -0.1566, -0.1600, -0.1530, -0.1549, -0.1535, -0.1505, -0.1528]])

In [29]:
dt_temp = torch.unsqueeze(test[0:200],dim = 1)

In [30]:
dt_temp

tensor([[[-0.1539, -0.1586, -0.1637,  ..., -0.1535, -0.1505, -0.1528]],

        [[-0.1539, -0.1586, -0.1637,  ..., -0.1535, -0.1505, -0.1528]],

        [[-0.1539, -0.1586, -0.1637,  ..., -0.1535, -0.1505, -0.1528]],

        ...,

        [[-0.1539, -0.1031, -0.1637,  ..., -0.1535, -0.1505, -0.1528]],

        [[-0.1539, -0.1586, -0.1637,  ..., -0.1535, -0.1505, -0.1528]],

        [[-0.1539, -0.1586, -0.1637,  ..., -0.1535, -0.1505, -0.1528]]])

In [31]:
model(dt_temp)

tensor([[[0.4894, 0.4871, 0.4888,  ..., 0.4036, 0.4944, 0.4924]],

        [[0.4894, 0.4871, 0.4888,  ..., 0.4929, 0.4944, 0.4924]],

        [[0.4894, 0.4871, 0.4888,  ..., 0.4929, 0.4944, 0.4924]],

        ...,

        [[0.5245, 0.4762, 0.4888,  ..., 0.4929, 0.4944, 0.4924]],

        [[0.4894, 0.4871, 0.4888,  ..., 0.4929, 0.4944, 0.4924]],

        [[0.4894, 0.4871, 0.4888,  ..., 0.4929, 0.4944, 0.4924]]],
       grad_fn=<ConvolutionBackward0>)